# Laboratorio 2, Statistical Learning 2

Silvio Urizar
4 de septiembre de  2023

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [2]:
Reviews = pd.read_csv('Restaurant.tsv',sep='\t')

In [3]:
Reviews.head()

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [4]:
texto = Reviews['Review'].values
sentimiento = Reviews['Liked'].values


## Separando etiquetas de texto

In [7]:
xTrain, xTeset, yTrain, yTest = train_test_split(texto,sentimiento,test_size = 0.25, random_state = 3312)

## Generación de tokens

In [8]:
tokens = Tokenizer(num_words = 6000, lower = True)
tokens.fit_on_texts(xTrain)

In [10]:
len_tok = len(tokens.word_index) + 1

## Secuencia para nuevos textos

In [11]:
seqTrain = tokens.texts_to_sequences(xTrain)
seqTest = tokens.texts_to_sequences(xTeset)

In [12]:
padseqTrain = pad_sequences(seqTrain,padding = 'post', maxlen = len(max(xTrain)))
padseqTest = pad_sequences(seqTest,padding = 'post', maxlen = len(max(xTrain)))


## Creando modelo

In [13]:
ModSent = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=len_tok,output_dim = 50, input_length= len(max(xTrain))),
    tf.keras.layers.LSTM(60),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

In [14]:
ModSent.compile(
    loss = tf.keras.losses.BinaryCrossentropy(),
    optimizer = 'adam',
    metrics = ['accuracy']
)

In [15]:
ModSent.fit(
    padseqTrain,
    yTrain,
    epochs = 20,
    validation_data = (padseqTest,yTest),
    batch_size = 50
)

Epoch 1/20
15/15 [==============================] - 5s 100ms/step - loss: 0.6936 - accuracy: 0.4947 - val_loss: 0.6930 - val_accuracy: 0.5160
Epoch 2/20
15/15 [==============================] - 1s 37ms/step - loss: 0.6939 - accuracy: 0.5080 - val_loss: 0.6941 - val_accuracy: 0.4840
Epoch 3/20
15/15 [==============================] - 0s 33ms/step - loss: 0.6933 - accuracy: 0.5053 - val_loss: 0.6940 - val_accuracy: 0.4840
Epoch 4/20
15/15 [==============================] - 1s 34ms/step - loss: 0.6933 - accuracy: 0.5053 - val_loss: 0.6936 - val_accuracy: 0.4840
Epoch 5/20
15/15 [==============================] - 0s 33ms/step - loss: 0.6933 - accuracy: 0.5053 - val_loss: 0.6935 - val_accuracy: 0.4840
Epoch 6/20
15/15 [==============================] - 0s 33ms/step - loss: 0.6933 - accuracy: 0.4853 - val_loss: 0.6933 - val_accuracy: 0.4840
Epoch 7/20
15/15 [==============================] - 0s 32ms/step - loss: 0.6934 - accuracy: 0.4760 - val_loss: 0.6931 - val_accuracy: 0.5160
Epoch 8/20
1

## Validación del modelo

In [16]:
loss, accuracy = ModSent.evaluate(padseqTest,yTest, verbose = False)
accuracy

0.7879999876022339

## Prueba con un texto

In [25]:
prueba = ['The food was very tasty. I loved the ravioli']
prseq = tokens.texts_to_sequences(prueba)
prseqpad = pad_sequences(prseq,padding = 'post',maxlen= len(max(xTrain)))

In [26]:
test = ModSent.predict(prseqpad)

1/1 [==============================] - 0s 39ms/step


In [29]:
print(test)

[[0.99303734]]


In [31]:
prueba2 = ['Nasty place to eat']
prseq2 = tokens.texts_to_sequences(prueba)
prseqpad2 = pad_sequences(prseq2,padding = 'post',maxlen= len(max(xTrain)))

In [32]:
otro = ModSent.predict(prseqpad2)

1/1 [==============================] - 0s 48ms/step


In [33]:
print(otro)

[[0.99303734]]
